In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model

In [18]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Load Network 15AB

In [19]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_1/models/B5R2_block5_15AB_1FC_3.h5' #model
model = load_model(model_dir)
height = width = model.input_shape[1]

In [20]:
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image/'#แก้
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Testdf.csv')#แก้
validation_dir = os.path.join(base_dir, 'test')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf.csv')#แก้
base_dir0 = '/media/tohn/SSD/Images/Image/'#แก้
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [21]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Testdf.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(1312, 22)
(1312, 22)
Normal:  (857, 22)
Abnormal:  (455, 22)


,Unnamed: 0,Unnamed: 0.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,Path Crop,Views,...,originalImage,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,filename
0,111,111,40,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,86.0,133.0,477.0,311.0,0.092664,0.148873,0.513514,0.346614,AB01 P1 C040.JPG
1,112,112,40,P2,P2,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,163.0,139.0,532.0,385.0,0.171698,0.154849,0.560377,0.428287,AB01 P2 C040.JPG
2,113,113,40,P4,P41,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,...,https://irisprodseatraining.blob.core.windows....,127.0,135.0,605.0,640.0,0.140316,0.150865,0.667984,0.711155,AB01 P4-1 C040.JPG
3,114,114,40,P5,P51,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-C,...,https://irisprodseatraining.blob.core.windows....,59.0,96.0,643.0,587.0,0.063462,0.107041,0.690385,0.653386,AB01 P5-1 C040.JPG
4,115,115,40,P3,P31,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,...,https://irisprodseatraining.blob.core.windows....,199.0,132.0,618.0,562.0,0.216374,0.146881,0.672515,0.625498,AB01 P3-1 C040.JPG


In [22]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Sub_class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 1312 validated image filenames belonging to 15 classes.
{0: 'AB01', 1: 'AB02', 2: 'AB03', 3: 'AB04', 4: 'AB05', 5: 'AB06', 6: 'AB07', 7: 'AB081', 8: 'AB082', 9: 'AB083', 10: 'AB09', 11: 'AB10', 12: 'AB11', 13: 'AB12', 14: 'Normal'}


In [23]:
conf_mat = pd.crosstab(train_df['Views'],train_df['Sub_class'],rownames=['5-FP Class'],colnames=['15AB Class'])
conf_mat

15AB Class,AB01,AB02,AB03,AB04,AB05,AB06,AB07,AB081,AB082,AB083,AB09,AB10,AB11,AB12,Normal
5-FP Class,,,,,,,,,,,,,,,
FP-A,84,109,42,84,34,59,2,22,21,8,0,0,0,0,596
FP-B,130,103,24,37,60,5,51,57,48,23,2,0,0,0,1063
FP-C,81,61,19,38,4,0,21,45,40,12,96,43,60,0,1015
FP-D,0,0,0,19,0,0,0,0,0,0,0,0,34,245,482
FP-E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,278


In [24]:
FP_ = []
for j in range(5) :
    FP = list(conf_mat.iloc[j,:][0:14])
    max_ = 11
    fp = []
    if j == 4:
        fp = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,1,1]
        FP_.append(fp)
    else:
        for i in FP:
    #         fp = []
            if i >= max_:
                a =1
                fp.append(a)
            elif i > 0 & i < max_ :
                a =0.5
                fp.append(a)                
            else:
                a =0
                fp.append(a)
        fp.append(1)
        FP_.append(fp)

In [25]:
FP_

[[1, 1, 1, 1, 1, 1, 0.5, 1, 1, 0.5, 0, 0, 0, 0, 1],
 [1, 1, 1, 1, 1, 0.5, 1, 1, 1, 1, 0.5, 0, 0, 0, 1],
 [1, 1, 1, 1, 0.5, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [26]:
fp

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [27]:
FP=np.array(FP_)
FP.shape

(5, 15)

In [28]:
# ['FPA','FPB','FPC','FPD','FPE']

In [29]:
df_ = pd.DataFrame(FP, columns = [ 'AB01','AB02','AB03', 'AB04', 'AB05', 'AB06', 'AB07', 'AB081', 'AB082', 'AB083', 'AB09', 'AB10', 'AB11', 'AB12', 'Normal'])
df_

,AB01,AB02,AB03,AB04,AB05,AB06,AB07,AB081,AB082,AB083,AB09,AB10,AB11,AB12,Normal
0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,0.5,0.0,0.0,0.0,0.0,1.0
1,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,0.5,0.0,0.0,0.0,1.0
2,1.0,1.0,1.0,1.0,0.5,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [30]:
print(FP)

[[1.  1.  1.  1.  1.  1.  0.5 1.  1.  0.5 0.  0.  0.  0.  1. ]
 [1.  1.  1.  1.  1.  0.5 1.  1.  1.  1.  0.5 0.  0.  0.  1. ]
 [1.  1.  1.  1.  0.5 0.  1.  1.  1.  1.  1.  1.  1.  0.  1. ]
 [0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. ]]


# Prediction

In [31]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model.predict([x])
    
    return result[0]

In [32]:
#Predict
pred_list = list()
prob_list = list()
pred_list_ = list()
img_path=dataframe['Path Crop'].tolist()
fp=dataframe['Views'].tolist()

In [33]:
for i in range(0,len(img_path)):
    print(i)
    if fp[i] =='FP-A':
        indx = 0
    elif fp[i] =='FP-B':
        indx = 1
    elif fp[i] =='FP-C':
        indx = 2
    elif fp[i] =='FP-D':
        indx = 3
    else :
        indx = 4
    predict_ = predict_image(img_path[i])
    prob = predict_*FP[indx]
    result = np.argmax(prob)
    pred_list_.append(labels[np.argmax(predict_)])
    pred_list.append(labels[result])
    prob_list.append(prob[result])
    
dataframe['category_old'] = pred_list_
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# เช็ค matrix

In [36]:
len(dataframe[dataframe['Prob']>0.5])

1293

In [40]:
dataframe[dataframe['Prob']>0.5]['Prob'].tolist()

[0.9583335518836975,
 0.9999228715896606,
 0.9323842525482178,
 0.7582891583442688,
 0.995622456073761,
 0.9778022766113281,
 0.9999997615814209,
 0.9990113973617554,
 0.9909105896949768,
 1.0,
 1.0,
 1.0,
 0.9999998807907104,
 0.9999988079071045,
 1.0,
 0.9999986886978149,
 1.0,
 1.0,
 0.9999997615814209,
 0.9998428821563721,
 0.9961572289466858,
 0.9365905523300171,
 0.9999901056289673,
 0.9998804330825806,
 0.9993011951446533,
 0.5764053463935852,
 0.9999973773956299,
 0.9999692440032959,
 1.0,
 0.9999979734420776,
 0.9999579191207886,
 0.988468587398529,
 0.6435301303863525,
 0.9996547698974609,
 0.985921323299408,
 0.9465771317481995,
 1.0,
 0.8655980229377747,
 1.0,
 0.991273820400238,
 0.9999998807907104,
 0.8907309174537659,
 0.9999957084655762,
 0.999935507774353,
 0.9521352052688599,
 1.0,
 0.9697938561439514,
 0.9999996423721313,
 0.999804675579071,
 0.9165840148925781,
 0.9963709115982056,
 0.5210676193237305,
 0.6714525818824768,
 0.9514471888542175,
 0.7339995503425598,
 

In [37]:
len(dataframe[dataframe['Prob']<=0.5])

19

In [43]:
dataframe[dataframe['Prob']<=0.5]

,Unnamed: 0,Unnamed: 0.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,Path Crop,Views,...,width,height,Rleft,Rtop,Rwidth,Rheight,filename,category_old,category,Prob
17,128,128,40,P1,P1,Abnormal,AB082,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,446.0,357.0,0.218107,0.057240,0.495885,0.384462,AB082_1 P1 C040.JPG,AB083,AB083,4.977874e-01
92,581,581,9,P4,P41,Abnormal,AB07,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,...,651.0,453.0,0.142342,0.156841,0.654054,0.503984,AB07 P4-1 C009.JPG,AB06,AB06,4.615622e-01
96,585,585,9,P1,P1,Abnormal,AB083,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,546.0,460.0,0.203125,0.089112,0.595703,0.511952,AB083 P1 C009.JPG,AB083,AB083,4.231368e-01
98,587,587,9,P5,P52,Abnormal,AB11,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-C,...,504.0,425.0,0.188755,0.160825,0.560241,0.470120,AB11 P5-2 C009.JPG,AB12,AB11,2.153872e-03
138,752,752,18,P5,P52,Abnormal,AB11,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-C,...,652.0,486.0,0.331839,0.210405,0.493274,0.540659,AB11 P5-2 C018.JPG,AB12,AB11,3.340903e-01
167,827,827,2,P6,P61,Abnormal,AB09,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-C,...,532.0,447.0,0.170341,0.166802,0.591182,0.494024,AB09 P6-1 C002 .JPG,AB12,AB09,5.291035e-02
184,1101,1101,1,P2,P2,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,418.0,452.0,0.191281,0.161742,0.417423,0.503030,AB02 P2 C001.JPG,AB10,AB01,2.031215e-04
196,1113,1113,1,P5,P52,Abnormal,AB04,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-C,...,676.0,507.0,0.072686,0.099116,0.686347,0.563636,AB04 P5-2 C001.JPG,AB12,AB11,1.436824e-05
284,1393,1393,30,P5,P52,Abnormal,AB11,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-C,...,725.0,620.0,0.014344,0.081144,0.807377,0.671315,AB11 P5-2 C030.JPG,AB12,AB11,3.848939e-01
293,1486,1486,65,P3,P31,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,...,603.0,623.0,0.049358,0.095076,0.670886,0.662626,AB02 P3-1 C065.JPG,AB02,AB02,4.833887e-01


In [42]:
dataframe[dataframe['Prob']<=0.5]['Prob']

17     4.977874e-01
92     4.615622e-01
96     4.231368e-01
98     2.153872e-03
138    3.340903e-01
167    5.291035e-02
184    2.031215e-04
196    1.436824e-05
284    3.848939e-01
293    4.833887e-01
322    4.120270e-01
328    6.690951e-06
354    1.726114e-03
372    4.628521e-01
378    5.747337e-09
387    4.328838e-02
392    2.472412e-07
393    6.735480e-03
986    4.974515e-01
Name: Prob, dtype: float64

In [51]:
FP

array([[1. , 1. , 1. , 1. , 1. , 1. , 0.5, 1. , 1. , 0.5, 0. , 0. , 0. ,
        0. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 0.5, 1. , 1. , 1. , 1. , 0.5, 0. , 0. ,
        0. , 1. ],
       [1. , 1. , 1. , 1. , 0.5, 0. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
        0. , 1. ],
       [0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ,
        1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
        1. , 1. ]])

In [60]:

    i =378
    if fp[i] =='FP-A':
        indx = 0
    elif fp[i] =='FP-B':
        indx = 1
    elif fp[i] =='FP-C':
        indx = 2
    elif fp[i] =='FP-D':
        indx = 3
    else :
        indx = 4
    
    predict_ = predict_image(img_path[378])
    result1 = np.argmax(predict_)
    print(predict_)
    print(result1, predict_[result1])
    
    prob = predict_*FP[indx]
    result = np.argmax(prob)
    print(prob)
    print(result, prob[result])
    print(FP[indx])

[6.6515871e-15 1.1036994e-14 2.0685417e-13 1.1634271e-13 2.6437983e-13
 2.8197515e-13 8.5978540e-14 2.4939352e-12 1.3908177e-10 6.6966161e-14
 3.0329862e-13 3.9195071e-13 5.7473368e-09 1.0000000e+00 3.5156672e-14]
13 1.0
[6.65158710e-15 1.10369943e-14 2.06854174e-13 1.16342713e-13
 1.32189913e-13 0.00000000e+00 8.59785401e-14 2.49393518e-12
 1.39081774e-10 6.69661605e-14 3.03298619e-13 3.91950713e-13
 5.74733683e-09 0.00000000e+00 3.51566717e-14]
12 5.7473368286764526e-09
[1.  1.  1.  1.  0.5 0.  1.  1.  1.  1.  1.  1.  1.  0.  1. ]


# Evaluation

## 15AB Class

In [ ]:
data_train = dataframe
#เช็คคลาสใน Predicted
pred_class = set(data_train['category'])
print('Predicted : ',len(pred_class))
print(pred_class)
#เช็คคลาสใน Actual
classe = set(data_train['Sub_class'])
print('Actual : ',len(classe))
print(classe)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
act = data_train['Sub_class'].array
pred = data_train['category'].array

cmat = confusion_matrix(act, pred)
print('classifier accuracy = {}%'.format((100.*np.trace(cmat))/(np.sum(cmat))))

#Marking the Confusion Matrix
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(act, pred))#performance

In [ ]:
#create CF 
data = {'Actual': act,'Predicted' : pred,}
df = pd.DataFrame(data, columns=['Actual','Predicted'])
conf_mat = pd.crosstab(df['Actual'],df['Predicted'],rownames=['Actual'],colnames=['Predicted'])

#Confusion matrix
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
cm = confusion_matrix(act, pred)

#plot Confusion matrix
import seaborn as sns
sns.set()
fig, ax = plt.subplots(figsize=(8, 5))

ax = sns.heatmap(conf_mat, annot=True, fmt="d", cmap="YlGnBu") #Blues,Oranges,Reds
ax.set_title('Confusion matrix',fontsize=20)
ax.set_ylabel('True label',fontsize=18)
ax.set_xlabel('Predicted label',fontsize=18)

## Abn&Nor Class

In [ ]:
act= data_train['Sub_class'].map({'AB12':1, 'AB04':1, 'AB05':1, 'Normal':0, 'AB02':1, 'AB11':1, 'AB082':1, 'AB06':1,'AB07':1, 'AB081':1, 'AB09':1, 'AB03':1, 'AB10':1, 'AB01':1, 'AB083':1}).values
pred = data_train['category'].map({'AB12':1, 'AB04':1, 'AB05':1, 'Normal':0, 'AB02':1, 'AB11':1, 'AB082':1, 'AB06':1,'AB07':1, 'AB081':1, 'AB09':1, 'AB03':1, 'AB10':1, 'AB01':1, 'AB083':1}).values
cmat = confusion_matrix(act, pred)
print('classifier accuracy = {}%'.format((100.*np.trace(cmat))/(np.sum(cmat))))

#Marking the Confusion Matrix
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(act, pred))#performance

In [ ]:
#create CF 
data = {'Actual': act,'Predicted' : pred,}
df = pd.DataFrame(data, columns=['Actual','Predicted'])
conf_mat = pd.crosstab(df['Actual'],df['Predicted'],rownames=['Actual'],colnames=['Predicted'])

#Confusion matrix
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
cm = confusion_matrix(act, pred)
TN, FP, FN, TP = confusion_matrix(act, pred).ravel()
print(TN, FP, FN, TP)
#plot Confusion matrix
import seaborn as sns
sns.set()
fig, ax = plt.subplots(figsize=(8, 5))

ax = sns.heatmap(conf_mat, annot=True, fmt="d", cmap="YlGnBu") #Blues,Oranges,Reds
ax.set_title('Confusion matrix',fontsize=20)
ax.set_ylabel('True label',fontsize=18)
ax.set_xlabel('Predicted label',fontsize=18)

# Top-3

In [ ]:
conf_mat = pd.crosstab(df0['Views'],df0['Sub_class'],rownames=['5-FP Class'],colnames=['15AB Class'])
conf_mat

In [ ]:
FP_ = []
for j in range(5) :
    FP = list(conf_mat.iloc[j,:][0:14])
    max_ = 11
    fp = []
    if j == 4:
        fp = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,1,1]
        FP_.append(fp)
    else:
        for i in FP:
    #         fp = []
            if i >= max_:
                a =1
                fp.append(a)
            elif i > 0 & i < max_ :
                a =0.5
                fp.append(a)                
            else:
                a =0
                fp.append(a)
        fp.append(1)
        FP_.append(fp)

In [ ]:
FP=np.array(FP_)
FP.shape

In [ ]:
print(FP)

In [ ]:
labels = {0: 'AB01', 1: 'AB02', 2: 'AB03', 3: 'AB04', 4: 'AB05', 5: 'AB06', 6: 'AB07', 7: 'AB081', 8: 'AB082', 9: 'AB083', 10: 'AB09', 11: 'AB10', 12: 'AB11', 13: 'AB12', 14: 'Normal'}

In [ ]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model.predict([x])
    
    return result[0]

In [ ]:
#Predict
pred_list = list()
prob_list = list()
pred_list_ = list()
img_path=dataframe['Path Crop'].tolist()
fp=dataframe['Views'].tolist()

In [ ]:
FP

In [ ]:
pred_list = list()
prob_list = list()

for i in range(0,len(img_path)):
    print(i)
    print(fp[i])
    if fp[i] =='FP-A':
        indx = 0
    elif fp[i] =='FP-B':
        indx = 1
    elif fp[i] =='FP-C':
        indx = 2
    elif fp[i] =='FP-D':
        indx = 3
    else :
        indx = 4
    predict_ = predict_image(img_path[i])
    result = predict_*FP[indx]
    re_class = list()
    re_prop = list()
    while len(re_class) <3:
#         print(i)
        maxx = np.argmax(result)
        re_class.append(labels[maxx])
        re_prop.append(result[maxx])
        result = np.where(result==result[maxx], 0, result)
    pred_list.append(re_class)
    prob_list.append(re_prop)

In [ ]:
dataframe['category_top3'] = pred_list
dataframe['Prob_top3'] = prob_list

In [ ]:
ress = list()
ress_class = list()
for i in range(0,len(dataframe)):
    if dataframe['Sub_class'][i] in dataframe['category_top3'][i]:
        res = dataframe['Sub_class'][i]
    else:
        res = dataframe['category'][i]
    ress.append(res)
dataframe['Pred_top3'] = ress

In [ ]:
Act_ = data_train['Sub_class'].tolist()

In [ ]:
cmat = confusion_matrix(Act_, ress)
print('classifier accuracy = {}%'.format((100.*np.trace(cmat))/(np.sum(cmat))))

#Marking the Confusion Matrix
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(Act_, ress))#performance

In [ ]:
#create CF 
data = {'Actual': Act_,'Predicted' : ress,}
df = pd.DataFrame(data, columns=['Actual','Predicted'])
conf_mat = pd.crosstab(df['Actual'],df['Predicted'],rownames=['Actual'],colnames=['Predicted'])

#Confusion matrix
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
cm = confusion_matrix(Act_, ress)

#plot Confusion matrix
import seaborn as sns
sns.set()
fig, ax = plt.subplots(figsize=(8, 5))

ax = sns.heatmap(conf_mat, annot=True, fmt="d", cmap="YlGnBu") #Blues,Oranges,Reds
ax.set_title('Confusion matrix',fontsize=20)
ax.set_ylabel('True label',fontsize=18)
ax.set_xlabel('Predicted label',fontsize=18)

In [ ]:
act= dataframe['Sub_class'].map({'AB12':1, 'AB04':1, 'AB05':1, 'Normal':0, 'AB02':1, 'AB11':1, 'AB082':1, 'AB06':1,'AB07':1, 'AB081':1, 'AB09':1, 'AB03':1, 'AB10':1, 'AB01':1, 'AB083':1}).values
pred= dataframe['Pred_top3'].map({'AB12':1, 'AB04':1, 'AB05':1, 'Normal':0, 'AB02':1, 'AB11':1, 'AB082':1, 'AB06':1,'AB07':1, 'AB081':1, 'AB09':1, 'AB03':1, 'AB10':1, 'AB01':1, 'AB083':1}).values

In [ ]:
cmat = confusion_matrix(act, pred)
print('classifier accuracy = {}%'.format((100.*np.trace(cmat))/(np.sum(cmat))))

#Marking the Confusion Matrix
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(act, pred))#performance

In [ ]:
#create CF 
data = {'Actual': act,'Predicted' : pred,}
df = pd.DataFrame(data, columns=['Actual','Predicted'])
conf_mat = pd.crosstab(df['Actual'],df['Predicted'],rownames=['Actual'],colnames=['Predicted'])

#Confusion matrix
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
cm = confusion_matrix(act, pred)
TN, FP, FN, TP = confusion_matrix(act, pred).ravel()
print(TN, FP, FN, TP)
#plot Confusion matrix
import seaborn as sns
sns.set()
fig, ax = plt.subplots(figsize=(8, 5))

ax = sns.heatmap(conf_mat, annot=True, fmt="d", cmap="YlGnBu") #Blues,Oranges,Reds
ax.set_title('Confusion matrix',fontsize=20)
ax.set_ylabel('True label',fontsize=18)

In [ ]:
# #3
# Test_df = pd.read_csv( '/home/yupaporn/CSV_file/result_BiTNet_FPScores.csv')#แก้
# Test_df['15AB_FPAct_3'] = dataframe['category']
# Test_df['Prob_FPAct_3'] = dataframe['Prob']
# Test_df.to_csv('/home/yupaporn/CSV_file/result_BiTNet_FPScores.csv', index=False)